# 고객요청의도 분석 후 활용하기 (OTP, 공인인증서)
## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )
## 민원(콜센터) 질의-응답 데이터 다운받기 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=98

# LangChain 라이브러리 설치

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf unstructured sentence-transformers langchain-community

In [ ]:
import pandas as pd

# 데이터 읽어오기

In [ ]:
json_file_path = '/content/민원(콜센터) 질의응답_금융보험_사고 및 보상 문의_Training.json'
df = pd.read_json(json_file_path)
df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,금융/보험,사고 및 보상 문의,A4075,고객,1,비밀번호 오류,,Q,인터넷뱅킹 로그인이 안돼요?,,,,"인터넷뱅킹, 로그인",인터넷뱅킹/ 금융서비스,"로그인,금융서비스"
1,금융/보험,사고 및 보상 문의,A4075,상담사,2,,비밀번호 오류,Q,,혹시 비밀번호를 잊어버리셨나요?,,,비밀번호,비밀번호/ 보안,"비밀번호,보안"
2,금융/보험,사고 및 보상 문의,A4075,고객,3,비밀번호 오류,,A,,,알고 있는 번호를 했는데 다 안된다고 나와요,,비밀번호,비밀번호/ 보안,"비밀번호,보안"
3,금융/보험,사고 및 보상 문의,A4075,상담사,4,,비밀번호 오류,Q,,비밀번호 5회이상 잘못 입력하셨나요?,,,"비밀번호, 5회",비밀번호/ 보안/ 5회/ 횟수,"5회,횟수"
4,금융/보험,사고 및 보상 문의,A4075,고객,5,비밀번호 오류,,A,,,어제 2번해서 안되서 오늘 다시 들어갔는데 안되요,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84027,금융/보험,사고 및 보상 문의,A32567,상담사,16,,분실 등록 사고,Q,,혹시 언제 카드를 잃어버린건가요?,,,카드,,카드
84028,금융/보험,사고 및 보상 문의,A32567,고객,17,분실 등록 사고,,A,,,그걸 모르겠어요,,,,
84029,금융/보험,사고 및 보상 문의,A32567,상담사,18,,분실 등록 사고,A,,,,어머니께 바로 분실신고 해달라고 하세요,분실신고,분실신고/유실신고,"분실신고,유실신고"
84030,금융/보험,사고 및 보상 문의,A32567,고객,19,분실 등록 사고,,Q,분실신고는 전화로 되는거죠?,,,,"분실신고, 전화",전화/연락,"전화,연락"


In [ ]:
len(df)

84032

In [ ]:
df.columns

Index(['도메인', '카테고리', '대화셋일련번호', '화자', '문장번호', '고객의도', '상담사의도', 'QA',
       '고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변', '개체명 ', '용어사전', '지식베이스'],
      dtype='object')

# 각 column별 unique한 값 확인하기

In [ ]:
도메인_리스트 = df['도메인'].unique()
for idx, 도메인 in enumerate(도메인_리스트):
  print(idx, 도메인)

0 금융/보험


In [ ]:
카테고리_리스트 = df['카테고리'].unique()
for idx, 카테고리 in enumerate(카테고리_리스트):
  print(idx, 카테고리)

0 사고 및 보상 문의


In [ ]:
대화셋일련번호_리스트 = df['대화셋일련번호'].unique()
for idx, 대화셋일련번호 in enumerate(대화셋일련번호_리스트):
  print(idx, 대화셋일련번호)

0 A4075
1 A4076
2 A4079
3 A4080
4 A4081
5 A4082
6 A4083
7 A4086
8 A4088
9 A4089
10 A4104
11 A4129
12 A4132
13 A4153
14 A4156
15 A4252
16 A4253
17 A4254
18 A4255
19 A4256
20 A4257
21 A4258
22 A4259
23 A4260
24 A4261
25 A4262
26 A4263
27 A4264
28 A4265
29 A4266
30 A4267
31 A4268
32 A4269
33 A4270
34 A4271
35 A4272
36 A4273
37 A4274
38 A4275
39 A4276
40 A4277
41 A4278
42 A4279
43 A4280
44 A4281
45 A4282
46 A4283
47 A4284
48 A4285
49 A4286
50 A4287
51 A4288
52 A4289
53 A4290
54 A4291
55 A4292
56 A4293
57 A4294
58 A4295
59 A4296
60 A4297
61 A4298
62 A4299
63 A4300
64 A4301
65 A4302
66 A4303
67 A4304
68 A4305
69 A4306
70 A4307
71 A4308
72 A4309
73 A4310
74 A4311
75 A4312
76 A4313
77 A4314
78 A4315
79 A4316
80 A4317
81 A4318
82 A4319
83 A4320
84 A4321
85 A4322
86 A4323
87 A4324
88 A4325
89 A4326
90 A5028
91 A5051
92 A5054
93 A5141
94 A5146
95 A5147
96 A5148
97 A5154
98 A5173
99 A5174
100 A5175
101 A5190
102 A5192
103 A5193
104 A5194
105 A5195
106 A5199
107 A5203
108 A5204
109 A5205
110 A5206


In [ ]:
화자_리스트 = df['화자'].unique()
for idx, 화자 in enumerate(화자_리스트):
  print(idx, 화자)

0 고객
1 상담사


In [ ]:
문장번호_리스트 = df['문장번호'].unique()
for idx, 문장번호 in enumerate(문장번호_리스트):
  print(idx, 문장번호)

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 26
26 27
27 28
28 29
29 30
30 31
31 32
32 33
33 34
34 35


In [ ]:
고객의도_리스트 = df['고객의도'].unique()
for idx, 고객의도 in enumerate(고객의도_리스트):
  print(idx, 고객의도)

0 비밀번호 오류
1 
2 오류해제
3 공인인증서
4 SMS문자서비스
5 고객등급
6 송금수수료
7 모바일뱅킹
8 분실문의
9 사고문의
10 분실등록
11 금융사고
12 금융사고신고
13 통장분실
14 OTP분실
15 OTP교체
16 수표분실
17 수표사고
18 현금카드분실
19 직불카드분실
20 ATM사고
21 보이스피싱
22 스미싱
23 개인정보유츌
24 착오입금
25 대포통장
26 현금카드범죄
27 펀드손해보상
28 분실신고
29 카드분실신고
30 보험증권
31 의료비청구
32 안내서비스
33 보험해약
34 보험청구
35 사고
36 보상문의
37 상해질병청구
38 배상청구
39 환급금청구
40 배상책임
41 자동차
42 질병상해청구
43 면책기간
44 치아보험
45 실손의료비
46 보험금청구
47 보험금문의
48 태아보험
49 보험금 청구
50 중복가입
51 일상배상책임보험
52 가전제품수리비
53 자동차사고
54 단독차사고
55 보상제외
56 시설소유자배상책임보험
57 침수피해보상처리
58 대리운전 사고처리
59 음주운전사고
60 교통사고
61 핸드폰파손수리
62 자기차량손해담보
63 통원의료비청구
64 보험금지급절차
65 보험금지급기일
66 보험금접수방법
67 보험금청구서류
68 현장확인
69 보험금비례보상
70 자기부담금
71 보험금청구기한
72 실손의료보험청구
73 의료심사
74 보험금부지급
75 고의사고
76 통지의무위반
77 해외외료비
78 여행자보험
79 자동차보험
80 통신영업사고담보
81 자동차사고보험청구
82 사망보험금
83 자동차보험청구
84 화재보험보상문의
85 정부보조금담보
86 정부지원금담보
87 임대보증담보
88 보증금보상담보
89 자동차보험면책
90 자동차보험담보
91 대물배상
92 자기신체사고보상
93 보험금계산청구
94 특별약관보상
95 장기화재보상청구
96 간병비보상문의
97 치아보장
98 질병보상문의
99 부동산잔금보상
100 카드가맹점담보
101 화재보험 보상문의
102 경매보상담보
103 보증상품문의
104 공탁금보

In [ ]:
상담사의도_리스트 = df['상담사의도'].unique()
for idx, 상담사의도 in enumerate(상담사의도_리스트):
  print(idx, 상담사의도)

0 
1 비밀번호 오류
2 오류해제
3 공인인증서
4 SMS문자서비스
5 고객등급
6 송금수수료
7 모바일뱅킹
8 분실문의
9 사고문의
10 분실등록
11 금융사고
12 금융사고신고
13 통장분실
14 OTP분실
15 OTP교체
16 수표분실
17 수표사고
18 현금카드분실
19 직불카드분실
20 ATM사고
21 보이스피싱
22 스미싱
23 개인정보유츌
24 착오입금
25 대포통장
26 현금카드범죄
27 펀드손해보상
28 분실신고
29 카드분실신고
30 보험증권
31 의료비청구
32 안내서비스
33 보험해약
34 보험청구
35 사고
36 보상문의
37 상해질병청구
38 배상청구
39 환급금청구
40 배상책임
41 자동차
42 질병상해청구
43 면책기간
44 치아보험
45 실손의료비
46 보험금청구
47 보험금문의
48 태아보험
49 보험금 청구
50 중복가입
51 일상배상책임보험
52 가전제품수리비
53 자동차사고
54 단독차사고
55 보상제외
56 시설소유자배상책임보험
57 침수피해보상처리
58 대리운전 사고처리
59 음주운전사고
60 교통사고
61 핸드폰파손수리
62 자기차량손해담보
63 통원의료비청구
64 보험금지급절차
65 보험금지급기일
66 보험금접수방법
67 보험금청구서류
68 현장확인
69 보험금비례보상
70 자기부담금
71 보험금청구기한
72 실손의료보험청구
73 의료심사
74 보험금부지급
75 고의사고
76 통지의무위반
77 해외외료비
78 여행자보험
79 자동차보험
80 통신영업사고담보
81 자동차사고보험청구
82 사망보험금
83 자동차보험청구
84 화재보험보상문의
85 정부보조금담보
86 정부지원금담보
87 임대보증담보
88 보증금보상담보
89 자동차보험면책
90 자동차보험담보
91 대물배상
92 자기신체사고보상
93 보험금계산청구
94 특별약관보상
95 장기화재보상청구
96 간병비보상문의
97 치아보장
98 질병보상문의
99 부동산잔금보상
100 카드가맹점담보
101 화재보험 보상문의
102 경매보상담보
103 공탁금보상담보
104 보석신

In [ ]:
QA_리스트 = df['QA'].unique()
for idx, QA in enumerate(QA_리스트):
  print(idx, QA)

0 Q
1 A


In [ ]:
고객질문_요청_리스트 = df['고객질문(요청)'].unique()
for idx, 고객질문_요청 in enumerate(고객질문_요청_리스트):
  print(idx, 고객질문_요청)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
14432 수술이나 입원 특약은 하나도 없나요?
14433 다른 보험은 다 있던데요?
14434 하지만 만기 환급금도 없잖아요?
14435 안녕하세요. 허혈성심장질환 보장이 무엇인지 궁금합니다.
14436 협심증과 심근경색 외에 다른 질환도 더 있나요?
14437 전부 1회씩만 주나요?
14438 수술비는요?
14439 네. 이것도 납입면제기능이 있나요?
14440 그럼 가입 후 1년 안에 걸렸을 때 50%만 주는 건요?
14441 수술비가 반복지원이 된다니 든든한데요?
14442 그런데 진단금은 왜 1회에 한하나요?
14443 발병률이 높은 질병은 보상을 줄이려고 하는건가요?
14444 아이 치아가 깨졌는데, 보험이 될까요?
14445 치아파절이 골절에 해당되는게 아니었나요?
14446 치아파절이 안들어가 있다면 실손청구더라도 못 받는단 건가요?
14447 치아보험이 있다면 다 들어가 있겠죠?
14448 치아보험이 2개이면 다 보상 받을 수 있는가요?
14449 치아보험은 비례보상하나요?
14450 치아보험 가입 직전 이가 빠졌는데, 보험처리가 안되나요?
14451 옆에 치아도 덩달아 흔들리는데, 그거는요?
14452 제 아이같은 경우 말고 보험 가입하자 마자 바로 보장 받을 수 있는 경우가 있나요?
14453 옆에 흔들리는 치아를 좀 더 놔뒀다 일이 커지면 질병으로 진단 받을 수 있는건가요?
14454 자세히 어떤 시기쯤 가능한지도 알 수 있을까요?
14455 요즘 코로나19로 걱정이 많은데, 이걸로 사망하면 질병사망인가요? 재해사망인가요?
14456 왜 그렇죠? 질병이잖아요.
14457 가입한 보험에  재해사망 특약이 가입되어 있다면 보상은 어떻게 받는거죠?
14458 특약이 없으면 일반사망보험금만 받겠네요?
14459 코로나19 처럼 1급 감염병에 속하면 재해로 인정되는건가요?
14460 손보사에서는 보장이 되나요?
14461 손보사에도 상해사망만 있으면 안되는거죠?
14462 또 다시 코

In [ ]:
상담사질문_요청_리스트 = df['상담사질문(요청)'].unique()
for idx, 상담사질문_요청 in enumerate(상담사질문_요청_리스트):
  print(idx, 상담사질문_요청)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1043 정지 요청하신 카드는 정지 되었습니다. 더 필요하신 사항은요?
1044 고객님처럼 카드를 분실 하셨을 경우 임시카드를 발급하실 수 있습니다. 발급 기간이1-3일 정도 소요되는데 발급 받으시겠습니까?
1045 안내를 해드릴까요?
1046 분실하셨으면 카드 정지를 먼저 처리해 드릴까요?
1047 진단비에 대하여 문의 주신 내용은 자세한 심사를 받아 보셔야 되는데 받아보시겠습니까?
1048 골절이신 부분에 대한 치료는 다 받으셨습니까?
1049 메모도 가능하신가요?
1050 통원치료 하셨습니까? 입원하셨습니까?
1051 병원비가 50만원 이하 입니까?
1052 50만원 이상이시면 추가로 제출하는 서류가 더 많습니다. 저희 보험사 어플은 사용해보셨나요?
1053 고객님 괜찮으신가요?
1054 사고경위를 말씀해주실수 있을까요?
1055 고객님 과실도 있으신가요?
1056 어떤 보험 청구인지 구체적으로 말씀해 주시겠습니까?
1057 알겠습니다. 지금 통화중인 번호로 보내드리면 되나요?
1058 치료는 잘 받으셨나요?
1059 치료비에 포함되냐고 물어보신건가요?
1060 필요한 서류가 몇가지 있는데요, 고객님 진료비는 얼마 나왔습니까?
1061 본인확인 후 도움드리겠습니다. 본인이신가요?
1062 필요한 서류 준비하셔서 저희 보험사로 보내주시면 되시는데요 담당 설계사 연락처는 아십니까? 
1063 진단서는 발급 받으셨습니까?
1064 지금 손 상태는 어떠신가요?
1065 필요한 서류를 제출하시면 되는데요, 병원비가 얼마 정도 되나요?
1066 혹시 휴대폰에 저희 보험사 어플 다운로드 받아 놓으셨나요?
1067 큰 사고이셨나요?
1068 고생 많으셨겠습니다. 치료는 잘 받으셨나요?
1069 입원치료를 하셨나요?
1070 본인이십니까?
1071 보험금 청구부터 하시는 것은 어떨까요?
1072 휴대폰에 저희 보험사 어플이 있으신가요?
1073 보험청구 메뉴 클릭하시면 암보험 청구라고 보이십니까?
1074 

In [ ]:
고객답변_리스트 = df['고객답변'].unique()
for idx, 고객답변 in enumerate(고객답변_리스트):
  print(idx, 고객답변)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2374 위점막하종양도 진단금 받을 수 있나요?
2375 같은 위에서 발생된 것인데, 암으로 만 진단이 내려야 하는건가요?
2376 그럼 암으로 판단은 어떤 암이든 다 가능한거죠?
2377 수술비를 받을 수 있을까 싶어서요
2378 발바닥에 티준 제거 시술을 받았어요 보장받을 수 있나 싶어서요
2379 사마귀아니고 병원에서 티눈이라고 했어요
2380 제가 이번에 너무 아파서 병원가서 검사를 해보니 이석증이라는 진단을 받았어요
2381 아,그래요.다행이네요
2382 아,네 잘알겠습니다, 퇴원할때 서류챙기도록 하겠습니다
2383 치료 받아서 처음보다는 좋아졌어요
2384 그래서 실비 청구가 가능 한지 궁금해서요
2385 1980년 00월00일입니다
2386 백내장수술을 다초점렌즈 삽입으로 하려고 하는데 실비 청구가 가능 한가 싶어서요
2387 ㅇㅇㅇㅇ년ㅇㅇ월ㅇㅇ일입니다
2388 아, 가능 하군요
2389 네 일단 잘 알겠습니다, 걱정말고 수술해야겠네요
2390 제가 계단에서 넘어져서 코가 부려져서 코수술을 했는데요, 그게 벌써 작년 12월인데요, 보험청구가 가능 한가 싶어서요.
2391 네 맞아요 오래 지나서 청구가 안될까요?
2392 ㅇㅇㅇ이구요, ㅇㅇㅇㅇ년ㅇ월,ㅇㅇ일입니다
2393 OOOO입니다
2394 아, 3년이내에요
2395 다행이 청구기간이 기네요
2396 네, 감사합니다, 청구하도록 하겠습니다
2397 제가 보험이 당사 보험뿐이여서요, 보험 청구가 걱정이 되서요.
2398 아, 되는군요
2399 아.3년이내요?
2400 네, 감사합니다
2401 네 맞아요 보험청구가 가능 한지 상담 좀 받으려구요
2402 아그래요?
2403 네, 서류챙겨서 청구 하도록 하겠습니다
2404 제가 운전자보험에 특약으로 가족일상생활배상책임보험이 가입되어 있는 상태입니다.
2405 전문가 확인 결과 우리집 화장실 배수구쪽에서 콘크리트에 금이가 그쪽으로 물이 새어나왔습니다. 보장이 가능 한가 싶어서요
2406 어느

In [ ]:
상담사답변_리스트 = df['상담사답변'].unique()
for idx, 상담사답변 in enumerate(상담사답변_리스트):
  print(idx, 상담사답변)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
14440 그러시군요. 염려가 되시겠습니다.
14441 네. 고객님께 좋은 소식이 있기를 바랍니다.
14442 통상 서류 접수 후 2주에서 1달정도 소요됩니다.
14443 아닙니다. 경부암일 경우는 암수술급여금에서만 지급됩니다.
14444 암을 제외한 각종 부인과질환으로 시술이 아닌 수술을 받았을 때 받을 수 있는 보장입니다.
14445 고객님 보험의 경우 2회까지 보장이 되게 되어있습니다.
14446 네 고객님. 말씀해주십시오.
14447 보험 계약 이후에 진단금 조정은 불가능합니다.
14448 네. 가입 당시에만 설정이 가능합니다.
14449 네. 암보험에서 암진단비는 천만원에서 5천만원 사이로 설정할 수 있게 되어 있습니다.
14450 네 그건 그렇겠죠.
14451 네 고객님. 각각의 보험에서 나오게 됩니다.
14452 네. 상담원 ㅇㅇㅇ이었습니다.
14453 네 고객님. 보험금을 부당하게 청구하는 것은 보험범죄에 해당합니다.
14454 금융감독원이나 당사의 보험범죄 신고센터, 손해보험협회의 보험범죄 신고센터를 통해 신고하실 수 있습니다.
14455 네 고객님. 실제 필요한 기간보다 훨씬 더 길게 진단서를 허위발급하여 치료를 받는 경우는 부당 보험금 청구로 볼 수 있습니다.
14456 네. 확실한 보험범죄에 해당하는 상황입니다.
14457 네. 포상금 제도를 운영하고 있습니다.
14458 신고자의 신원이 확인되지 않는 경우는 포상에서 제외됩니다.
14459 네 그렇습니다. 신고자의 신상정보는 비밀로 보장됩니다.
14460 보험범죄 혐의자, 일시, 보험범죄내용, 해당 보험회사, 보험계약내용, 보험종목 등을 알아놓으시면 좋습니다.
14461 추가적으로 문의사항 있으시면 언제든 고객센터로 연락주십시오.
14462 네 고객님. 사고로 인한 큰 피해가 없으셨길 바랍니다.
14463 네. 동승자분에 대한 보상은 고객님이 가입하신 보험과 고객님 뒷차가 가입한 보험이 비례분담하여 내게 됩니다.
14464 

In [ ]:
개체명_리스트 = df['개체명 '].unique()
for idx, 개체명 in enumerate(개체명_리스트):
  print(idx, 개체명)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
24411 직장내 따돌림, SMS, 문자
24412 근로기준법, 제 76조
24413 근로자, 업무, 직장
24414 입원치료, 산재보험, 결제
24415 산재지정병원, 치료
24416 비용, 청구
24417 산재보험, 비급여, 지급
24418 업무상 재해, 치료, 기간, 비용, 지급
24419 비타민 주사, 지급
24420 입원치료, 입원, 기간, 보험, 지급
24421 입원, 기간
24422 치료, 연장, 제출
24423 불이익, 치료, 청구
24424 산재의료기관, 치료
24425 보상금, 사고, 지급
24426 장해급여, 지급
24427 치료, 상해, 손해
24428 산재보험, 급여
24429 근로자, 기관, 보험, 업무, 질병
24430 급여, 소득, 지급
24431 간병급여
24432 근로자, 재해, 급여, 지급
24433 유족급여, 지급
24434 근로자, 재해, 사망
24435 근로자, 재해, 지급, 사망
24436 120일, 1일, 지급
24437 재해, 치료, 지급
24438 직업재활급여
24439 장해급여, 근로자, 직업, 지급
24440 산재보험, 치료, 신청
24441 근로복지공단, 근로자, 신청
24442 사업주, 신청
24443 급여, 신청, 청구
24444 산재지정병원, 의료비, 치료, 급여, 신청, 지급, 병원
24445 산재지정병원, 치료비, 치료, 결제, 청구
24446 택배기사
24447 산재보험, 택배기사, 근로자, 보장, 가입
24448 산재보험료, 사업주, 근로자, 가입, 납입
24449 사업주, 50%, 근로자
24450 근로자, 납입
24451 100%, 사업주
24452 70%, 지급
24453 2년, 연금, 지급
24454 산재보험, 통원비, 급여
24455 통원비, 급여
24456 보조기, 급여, 비용
24457 보조기, 급여
24458 급여, 비용
24459 검사비용, 급여
24460 급여, 비용, 병원
24461 처치비, 급여
24462 산재보

In [ ]:
용어사전_리스트 = df['용어사전'].unique()
for idx, 용어사전 in enumerate(용어사전_리스트):
  print(idx, 용어사전)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
23858 렌트카 비용/비용, 자동차 사고/사고
23859 피해자/피해/사람
23860 9:1/비율
23861 10%/비율, 비용/지출
23862 수리/행위, 비용/경비
23863 보험사/기관, 수리/행위, 비용/지출
23864 금요일/요일, 수리/행위, 회사/장소
23865 월요일/요일, 주말/영업시간
23866 금요일/요일, 보험사/기관, 연장/기간
23867 보험사/기관, 수리/행위, 거절/취소
23868 렌트카 비용/비용, 보험사/기관, 수리/행위, 거절/취소, 지불/결제
23869 금감원/기관, 민원/분쟁
23870 홈페이지/사이트, 금감원/기관
23871 m.fss.or.kr/홈페이지
23872 보험증권/서류, 분실/사고
23873 재발급/관리, 사본/서류
23874 보험증권/서류, 원본/서류
23875 공인인증서/인증수단, 홈페이지/사이트, 로그인/접속, 출력/인쇄, 회사/장소
23876 발급/발행, 비용/경비
23877 무료/돈, 발급/발행, 비용/경비
23878 분실/사고, 약관/계약
23879 약관/계약, 우편/채널
23880 ㅇㅇ동/주소, ㅇㅇ시/주소, ㅇ구/주소
23881 ㅇㅇㅇ호/주소, 아파트/주거형태
23882 사본/서류, 약관/계약
23883 ㅇㅇㅇ호/주소, ㅇㅇ동/주소, ㅇㅇ시/주소, 아파트/주거형태, ㅇ구/주소
23884 소요/필요
23885 약관/계약, 소요/필요
23886 일반암/암, 특약/보험 계약, 보장/보험 보장
23887 생식기암/암
23888 방광암, 전립선암, 유방암, 자궁암/암/병
23889 납입기간/기간, 만기/기간
23890 100세/연령, 5년/기간, 납입/지출, 만기/기간
23891 보험금/돈, 지급/비용, 수술/의료행위
23892 500만원/금액, 일반암/암, 수술비/돈, 1회/횟수
23893 1일/기간, 만원/금액
23894 후유장해/장해, 보장/보험 보장
23895 후유장해/장해, 5천만원/금액, 80%/비율, 진단금/돈, 지급/비용
23

In [ ]:
지식베이스_리스트 = df['지식베이스'].unique()
for idx, 지식베이스 in enumerate(지식베이스_리스트):
  print(idx, 지식베이스)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
19917 양방의사,기관
19918 의료기관,계약내용
19919 환자수용,법률용어
19920 실손,등록
19921 의료,금융상품
19922 보상한도,보상
19923 수급권자,의료제도
19924 실손보험,금융상품금액
19925 5%,비율
19926 지원책,정부시책
19927 가입,보험상품금액
19928 납입,보험자격
19929 할인,보험상품금액
19930 보험,비율
19931 가족,기관
19932 질병,금융상품
19933 할인,관계
19934 상해,금융서비스
19935 본인부담상한제,의료제도
19936 건강보험,보험자격
19937 상한액,비용
19938 580만원,금액
19939 본인부담금,기관
19940 본인부담상한액,비용
19941 변경,기관
19942 비율,금융상품금액
19943 통지의무,기관
19944 계약,기관
19945 형사합의지원금,비용
19946 비례,보장
19947 실손,비용
19948 가입,이중
19949 가입,날짜
19950 질병입원의료비,비용
19951 체척기간,법률용어
19952 법률,법률용어
19953 부담금,보장
19954 분실,기관
19955 의료,보장
19956 의료,날짜
19957 피보험자,보장
19958 상해,원인
19959 차량,탑승
19960 이해,깨달음
19961 배우자,보장
19962 조항,보장
19963 부모,보장
19964 동승자,상쇠
19965 한도,규모
19966 대중교통,보장
19967 보상,적용
19968 해당,보증
19969 해당,조건
19970 사고발생지,조건
19971 공휴일,내포
19972 신주말,휴일
19973 표준시,조건
19974 지급금액,돈
19975 진단확정,처리
19976 화상,재해
19977 열상,내포
19978 보험금,처리
19979 1회,처리
19980 보험금,재해
19981 시운전,보상
19982 심재성,급수
19983 2도,급수
19984 병원,보험자격
19985 특약,재해
19986 보험료,처리
19987 법정대리인,보

# '고객의도' 종류 확인하기

In [ ]:
고객의도_리스트 = df['고객의도'].unique()
for idx, 고객의도 in enumerate(고객의도_리스트):
  print(idx, 고객의도)

0 비밀번호 오류
1 
2 오류해제
3 공인인증서
4 SMS문자서비스
5 고객등급
6 송금수수료
7 모바일뱅킹
8 분실문의
9 사고문의
10 분실등록
11 금융사고
12 금융사고신고
13 통장분실
14 OTP분실
15 OTP교체
16 수표분실
17 수표사고
18 현금카드분실
19 직불카드분실
20 ATM사고
21 보이스피싱
22 스미싱
23 개인정보유츌
24 착오입금
25 대포통장
26 현금카드범죄
27 펀드손해보상
28 분실신고
29 카드분실신고
30 보험증권
31 의료비청구
32 안내서비스
33 보험해약
34 보험청구
35 사고
36 보상문의
37 상해질병청구
38 배상청구
39 환급금청구
40 배상책임
41 자동차
42 질병상해청구
43 면책기간
44 치아보험
45 실손의료비
46 보험금청구
47 보험금문의
48 태아보험
49 보험금 청구
50 중복가입
51 일상배상책임보험
52 가전제품수리비
53 자동차사고
54 단독차사고
55 보상제외
56 시설소유자배상책임보험
57 침수피해보상처리
58 대리운전 사고처리
59 음주운전사고
60 교통사고
61 핸드폰파손수리
62 자기차량손해담보
63 통원의료비청구
64 보험금지급절차
65 보험금지급기일
66 보험금접수방법
67 보험금청구서류
68 현장확인
69 보험금비례보상
70 자기부담금
71 보험금청구기한
72 실손의료보험청구
73 의료심사
74 보험금부지급
75 고의사고
76 통지의무위반
77 해외외료비
78 여행자보험
79 자동차보험
80 통신영업사고담보
81 자동차사고보험청구
82 사망보험금
83 자동차보험청구
84 화재보험보상문의
85 정부보조금담보
86 정부지원금담보
87 임대보증담보
88 보증금보상담보
89 자동차보험면책
90 자동차보험담보
91 대물배상
92 자기신체사고보상
93 보험금계산청구
94 특별약관보상
95 장기화재보상청구
96 간병비보상문의
97 치아보장
98 질병보상문의
99 부동산잔금보상
100 카드가맹점담보
101 화재보험 보상문의
102 경매보상담보
103 보증상품문의
104 공탁금보

# 고객의도가 'OTP'와 관련된 데이터만 필터링하기

In [ ]:
OTP_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if 'OTP' in 의도]
OTP_키워드_포함_고객의도_리스트

['OTP분실',
 'OTP교체',
 'OTP오류',
 '타행 OTP 등록',
 'OTP 분실',
 'OTP 보안카드 변경',
 'OTP 보안카드 이체한도',
 'OTP 오류']

In [ ]:
filtered_df = df[(df['고객의도'] == 'OTP분실') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
976,금융/보험,사고 및 보상 문의,A4285,고객,1,OTP분실,,Q,제가 OTP를 분실했는데요.,,,,"OTP, 분실",OTP/보안매체/ 분실/사고,"분실,사고"
8092,금융/보험,사고 및 보상 문의,A6079,고객,1,OTP분실,,Q,OTP발생기 분실했는데 어떻게 하나요?,,,,OTP발생기,OTP발생기/서비스,"OTP발생기,서비스"
64830,금융/보험,사고 및 보상 문의,A28443,고객,1,OTP분실,,Q,안녕하세요 S은행이죠?,,,,은행,은행/ 금융기관,"은행,금융기관"
67348,금융/보험,사고 및 보상 문의,A28679,고객,1,OTP분실,,Q,OTP발생기 분실했는데 어떻게 하나요?,,,,OTP발생기,OTP발생기/서비스,"OTP발생기,서비스"
71153,금융/보험,사고 및 보상 문의,A29168,고객,1,OTP분실,,Q,OTP발생기 분실했는데 어떻게 하나요?,,,,OTP발생기,OTP발생기/서비스,"OTP발생기,서비스"


In [ ]:
filtered_df = df[(df['고객의도'] == 'OTP교체') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
996,금융/보험,사고 및 보상 문의,A4286,고객,1,OTP교체,,Q,"아, 저 이체하려는데 OTP가 안돼요.",,,,"이체, OTP",이체/금융서비스/ OTP/보안매체,"OTP,보안매체"


In [ ]:
filtered_df = df[(df['고객의도'] == 'OTP오류') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
8797,금융/보험,사고 및 보상 문의,A7052,고객,1,OTP오류,,Q,OTP번호 몇번 잘못입력하면 거래 못하게 되나요?,,,,"OTP, 잘못, 입력, 거래","OTP/1회용비밀번호생성기, 입력/기록, 거래/교환","잘못,교환"


In [ ]:
filtered_df = df[(df['고객의도'] == '타행 OTP 등록') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
45648,금융/보험,사고 및 보상 문의,A18256,고객,1,타행 OTP 등록,,Q,타행에서 발급받은 OTP기기를 등록하려고 하는데 잘 안됩니다.,,,,"OTP, 타행, 발급, 등록, 기기","OTP/보안, 타행/은행, 발급/발행, 등록/등록, 기기/기기","타행,기기"
67828,금융/보험,사고 및 보상 문의,A28703,고객,1,타행 OTP 등록,,Q,타행에서 발급받은 OTP기기를 등록하려고 하는데 잘 안됩니다.,,,,"OTP, 타행, 발급, 등록, 기기","OTP/보안, 타행/은행, 발급/발행, 등록/등록, 기기/기기","타행,기기"
71633,금융/보험,사고 및 보상 문의,A29192,고객,1,타행 OTP 등록,,Q,타행에서 발급받은 OTP기기를 등록하려고 하는데 잘 안됩니다.,,,,"OTP, 타행, 발급, 등록, 기기","OTP/보안, 타행/은행, 발급/발행, 등록/등록, 기기/기기","타행,기기"


In [ ]:
filtered_df = df[(df['고객의도'] == 'OTP 분실') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
46228,금융/보험,사고 및 보상 문의,A18526,고객,1,OTP 분실,,Q,OTP를 잃어버렸어요.,,,,OTP,OTP/보안,"OTP,보안"


In [ ]:
filtered_df = df[(df['고객의도'] == 'OTP 보안카드 변경') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
46268,금융/보험,사고 및 보상 문의,A18528,고객,1,OTP 보안카드 변경,,Q,안녕하세요. 지금 OTP를 사용중인데 이걸 제가 가지고 있는 보안카드로 변경할 수 ...,,,,"보안카드, OTP, 변경","보안카드/보안, OTP/보안, 변경/행위","OTP,행위"


In [ ]:
filtered_df = df[(df['고객의도'] == 'OTP 보안카드 이체한도') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
46288,금융/보험,사고 및 보상 문의,A18529,고객,1,OTP 보안카드 이체한도,,Q,OTP에서 보안카드로 바꾸면 이체 한도가 줄어드나요?,,,,"보안카드, OTP, 한도, 이체","보안카드/보안, OTP/보안, 한도/대출, 이체/거래","OTP,거래"


In [ ]:
filtered_df = df[(df['고객의도'] == 'OTP 오류') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
46708,금융/보험,사고 및 보상 문의,A18678,고객,1,OTP 오류,,Q,제 OTP가 에러가 났는데 어떻게 하죠?,,,,"OTP,에러","OTP-보안인증수단,에러/오류","에러,오류"


# 고객의도가 '공인인증서'와 관련된 데이터만 필터링하기

In [ ]:
공인인증서_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '공인인증서' in 의도]
공인인증서_키워드_포함_고객의도_리스트

['공인인증서',
 '공인인증서오류',
 '스마트폰 공인인증서 복사',
 '공인인증서 자택 직장병용',
 '증권사 공인인증서',
 '공인인증서 분실',
 '일반 공인인증서와 범용 공인인증서 병용',
 '공인인증서 USB 분실',
 '공인인증서 폐지']

In [ ]:
filtered_df = df[(df['고객의도'] == '공인인증서') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
20,금융/보험,사고 및 보상 문의,A4076,고객,1,공인인증서,,Q,미성년 자녀도 인터넷뱅킹 가능한가요?,,,,"미성년자, 인터넷뱅킹, 공인인증서",미성년자/청소년/나이/ 인터넷뱅킹/온라인뱅킹/스마트뱅킹/ 공인인증서/전자신분증,"인터넷뱅킹,전자신분증"
40,금융/보험,사고 및 보상 문의,A4079,고객,1,공인인증서,,Q,공인인증서를 온라인에서 발급 가능한가요?,,,,"공인인증서, 발급, 온라인",발급/발부/ 온라인/인터넷,"발급,인터넷"
81,금융/보험,사고 및 보상 문의,A4081,고객,1,공인인증서,,Q,안녕하세요. 공인인증서 관련해서 문의좀 드리려구요,,,,"공인인증서, 관련, 질문",공인인증서/인증서,"관련,인증서"
141,금융/보험,사고 및 보상 문의,A4086,고객,1,공인인증서,,Q,기업인터넷 뱅킹의 공인인중서 암호 오류가 납니다. 어떻게 해야 하나요?,,,,"기업인터넷뱅킹, 공인인증서, 암호, 오류",기업/기관/공인인증서/암호/오류,"공인인증서,오류"
38386,금융/보험,사고 및 보상 문의,A13159,고객,1,공인인증서,,Q,안녕하세요 공인인증서 재발급을 받으려고 합니다.,,,,"공인인증서, 재발급",공인인증서/인증서,"재발급,인증서"
83084,금융/보험,사고 및 보상 문의,A32085,고객,1,공인인증서,,Q,공인인증서를 온라인에서 발급 가능한가요?,,,,"공인인증서, 발급, 온라인","발급/발부, 온라인/인터넷, 공인인증서/증명서","발급,증명서"
83104,금융/보험,사고 및 보상 문의,A32087,고객,1,공인인증서,,A,,,안녕하세요. 공인인증서 관련해서 문의좀 드리려구요,,"공인인증서, 관련, 질문","공인인증서/인증서, 관련/연관, 질문/질의","관련,질의"
83144,금융/보험,사고 및 보상 문의,A32092,고객,1,공인인증서,,Q,기업인터넷 뱅킹의 공인인중서 암호 오류가 납니다. 어떻게 해야 하나요?,,,,"기업인터넷뱅킹, 공인인증서, 암호, 오류","기업인터넷뱅킹/인터넷뱅킹, 암호/패스워드, 오류/에러","공인인증서,에러"


In [ ]:
filtered_df = df[(df['고객의도'] == '공인인증서오류') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
8654,금융/보험,사고 및 보상 문의,A7045,고객,1,공인인증서오류,,Q,공인인증서를 로그인하려는데 안보여요?,,,,"공인인증서, 로그인","공인인증서/공동인증서, 로그인/접속","로그인,접속"


In [ ]:
filtered_df = df[(df['고객의도'] == '스마트폰 공인인증서 복사') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
45728,금융/보험,사고 및 보상 문의,A18470,고객,1,스마트폰 공인인증서 복사,,Q,스마트폰에 있는 인증서를 가지고 사용하고 싶은데요.,,,,"스마트폰, 인증서","스마트폰/기기, 인증서/보안인증","인증서,보안인증"


In [ ]:
filtered_df = df[(df['고객의도'] == '공인인증서 자택 직장병용') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
45748,금융/보험,사고 및 보상 문의,A18475,고객,1,공인인증서 자택 직장병용,,Q,집과 회사 양쪽에서 인터넷 뱅킹이 사용 가능할까요?,,,,"인터넷 뱅킹, 회사, 집","인터넷 뱅킹/금융서비스, 회사/장소, 집/장소","회사,장소"


In [ ]:
filtered_df = df[(df['고객의도'] == '증권사 공인인증서') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
45808,금융/보험,사고 및 보상 문의,A18483,고객,1,증권사 공인인증서,,Q,안녕하세요. 인증서에 대해서 좀 여쭙고자 합니다.,,,,인증서,인증서/보안인증,"인증서,보안인증"


In [ ]:
filtered_df = df[(df['고객의도'] == '공인인증서 분실') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
45828,금융/보험,사고 및 보상 문의,A18486,고객,1,공인인증서 분실,,Q,컴퓨터를 포맷하고 나서 공인인증서가 없어졌어요.,,,,"공인인증서, 컴퓨터, 포맷","공인인증서/공인인증서, 컴퓨터/전자기기, 포맷/컴퓨터",컴퓨터
46628,금융/보험,사고 및 보상 문의,A18629,고객,1,공인인증서 분실,,Q,공인인증서를 분실했어요.,,,,"공인인증서, 분실","공인인증서/인증수단, 분실/사고","분실,사고"


In [ ]:
filtered_df = df[(df['고객의도'] == '일반 공인인증서와 범용 공인인증서 병용') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
45988,금융/보험,사고 및 보상 문의,A18507,고객,1,일반 공인인증서와 범용 공인인증서 병용,,Q,일반 공인인증서랑 범용 공인인증서를 같이 사용할 수 있는지 문의드립니다.,,,,"일반 공인인증서, 범용 공인인증서, 문의","일반 공인인증서/공인인증서, 범용 공인인증서/공인인증서, 문의/문의","범용공인인증서,문의"


In [ ]:
filtered_df = df[(df['고객의도'] == '공인인증서 USB 분실') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
46348,금융/보험,사고 및 보상 문의,A18539,고객,1,공인인증서 USB 분실,,Q,공인인증서가 담긴 USB를 분실했어요.,,,,"공인인증서, USB, 분실","공인인증서/공인인증서, USB/저장매체, 분실/사고","USB,사고"


In [ ]:
filtered_df = df[(df['고객의도'] == '공인인증서 폐지') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
46568,금융/보험,사고 및 보상 문의,A18626,고객,1,공인인증서 폐지,,Q,공인인증서 폐지는 언제부터 실시되나요?,,,,"공인인증서, 폐지","공인인증서/인증수단, 폐지/취소","폐지,취소"


# 사용자 질문으로부터 '고객요청의도' 추출하기

##  OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "여러분의_OPENAI_API_KEY"

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_KEY)

# create_tagging_chain으로 고객 요청의도 추출하기

In [ ]:
from langchain.chains import create_tagging_chain

# 고객요청의도 추출을 위한 schema 설정
schema = {
    "properties": {
        "고객요청의도": {
            "type": "string",
            "enum": ['OTP',
                     '공인인증서'
            ]
        }
    }
}
extraction_chain = create_tagging_chain(schema, llm)

In [ ]:
chain_run_result = extraction_chain.run("공인인증서 폐지는 언제부터 실시되나요?")
chain_run_result

{'고객요청의도': '공인인증서'}

In [ ]:
chain_run_result = extraction_chain.run("제 OTP가 에러가 났는데 어떻게 하죠?")
chain_run_result

{'고객요청의도': 'OTP'}

In [ ]:
chain_run_result = extraction_chain.run("안녕하세요 S은행이죠?")
chain_run_result

{}

# Vectorstore에 저장하기 위한 데이터 정제하기

In [ ]:
공인인증서_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '공인인증서' in 의도]
공인인증서_키워드_포함_고객의도_리스트

['공인인증서',
 '공인인증서오류',
 '스마트폰 공인인증서 복사',
 '공인인증서 자택 직장병용',
 '증권사 공인인증서',
 '공인인증서 분실',
 '일반 공인인증서와 범용 공인인증서 병용',
 '공인인증서 USB 분실',
 '공인인증서 폐지']

In [ ]:
OTP_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if 'OTP' in 의도]
OTP_키워드_포함_고객의도_리스트

['OTP분실',
 'OTP교체',
 'OTP오류',
 '타행 OTP 등록',
 'OTP 분실',
 'OTP 보안카드 변경',
 'OTP 보안카드 이체한도',
 'OTP 오류']

In [ ]:
최종_필터링용_고객의도_리스트 = 공인인증서_키워드_포함_고객의도_리스트 + OTP_키워드_포함_고객의도_리스트
최종_필터링용_고객의도_리스트

['공인인증서',
 '공인인증서오류',
 '스마트폰 공인인증서 복사',
 '공인인증서 자택 직장병용',
 '증권사 공인인증서',
 '공인인증서 분실',
 '일반 공인인증서와 범용 공인인증서 병용',
 '공인인증서 USB 분실',
 '공인인증서 폐지',
 'OTP분실',
 'OTP교체',
 'OTP오류',
 '타행 OTP 등록',
 'OTP 분실',
 'OTP 보안카드 변경',
 'OTP 보안카드 이체한도',
 'OTP 오류']

In [ ]:
len(최종_필터링용_고객의도_리스트)

17

In [ ]:
filtered_df = df[df['고객의도'].isin(최종_필터링용_고객의도_리스트)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
20,금융/보험,사고 및 보상 문의,A4076,고객,1,공인인증서,,Q,미성년 자녀도 인터넷뱅킹 가능한가요?,,,,"미성년자, 인터넷뱅킹, 공인인증서",미성년자/청소년/나이/ 인터넷뱅킹/온라인뱅킹/스마트뱅킹/ 공인인증서/전자신분증,"인터넷뱅킹,전자신분증"
22,금융/보험,사고 및 보상 문의,A4076,고객,3,공인인증서,,A,,,아니요.,,,,
24,금융/보험,사고 및 보상 문의,A4076,고객,5,공인인증서,,Q,준비할 서류가 뭐죠?,,,,"어떤, 서류",,서류
26,금융/보험,사고 및 보상 문의,A4076,고객,7,공인인증서,,Q,"아, 학생증도 가능한가요?",,,,학생증,학생증/신분증,"학생증,신분증"
28,금융/보험,사고 및 보상 문의,A4076,고객,9,공인인증서,,A,,,생년월일만 나와있어요.,,생년월일,생년월일/나이,"생년월일,나이"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83155,금융/보험,사고 및 보상 문의,A32092,고객,12,공인인증서,,Q,어떤 형식으로 암호를 만들어야 하나요?,,,,"형식, 암호","형식/스타일, 암호/비밀번호","암호,비밀번호"
83157,금융/보험,사고 및 보상 문의,A32092,고객,14,공인인증서,,A,,,네 감사합니다.,,,,
83159,금융/보험,사고 및 보상 문의,A32092,고객,16,공인인증서,,Q,인증서 재발급 경로는 어떻게 되나요?,,,,"인증서, 재발급, 경로",경로/루트,"재발급,루트"
83161,금융/보험,사고 및 보상 문의,A32092,고객,18,공인인증서,,Q,인증서 재발급시 필요한 정보들이 있을까요?,,,,"인증서, 재발급,정보","인증서/증명서, 정보/데이터","재발급,데이터"


# 중복 데이터 제거

In [ ]:
# Identify duplicates by considering all columns except '대화셋일련번호'
duplicates = filtered_df.duplicated(subset=filtered_df.columns.difference(['대화셋일련번호']), keep=False)

# Remove duplicates
df_cleaned = filtered_df[~duplicates]
df_cleaned

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
20,금융/보험,사고 및 보상 문의,A4076,고객,1,공인인증서,,Q,미성년 자녀도 인터넷뱅킹 가능한가요?,,,,"미성년자, 인터넷뱅킹, 공인인증서",미성년자/청소년/나이/ 인터넷뱅킹/온라인뱅킹/스마트뱅킹/ 공인인증서/전자신분증,"인터넷뱅킹,전자신분증"
22,금융/보험,사고 및 보상 문의,A4076,고객,3,공인인증서,,A,,,아니요.,,,,
24,금융/보험,사고 및 보상 문의,A4076,고객,5,공인인증서,,Q,준비할 서류가 뭐죠?,,,,"어떤, 서류",,서류
26,금융/보험,사고 및 보상 문의,A4076,고객,7,공인인증서,,Q,"아, 학생증도 가능한가요?",,,,학생증,학생증/신분증,"학생증,신분증"
28,금융/보험,사고 및 보상 문의,A4076,고객,9,공인인증서,,A,,,생년월일만 나와있어요.,,생년월일,생년월일/나이,"생년월일,나이"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83147,금융/보험,사고 및 보상 문의,A32092,고객,4,공인인증서,,Q,3회 입력오류라고 뜨는데 어떻게 하죠?,,,,"3회, 입력, 오류","3회/홧수, 입력/인풋, 오류/에러","입력,에러"
83153,금융/보험,사고 및 보상 문의,A32092,고객,10,공인인증서,,Q,인증서 암호는 몇자리 수 이상으로 정해진게 있나요?,,,,"인증서, 암호, 이상, 수","이상/위, 수/숫자","암호,숫자"
83155,금융/보험,사고 및 보상 문의,A32092,고객,12,공인인증서,,Q,어떤 형식으로 암호를 만들어야 하나요?,,,,"형식, 암호","형식/스타일, 암호/비밀번호","암호,비밀번호"
83159,금융/보험,사고 및 보상 문의,A32092,고객,16,공인인증서,,Q,인증서 재발급 경로는 어떻게 되나요?,,,,"인증서, 재발급, 경로",경로/루트,"재발급,루트"


In [ ]:
대화셋_set = set()
for count, (index, row) in enumerate(df_cleaned.iterrows()):
    print(count, row['대화셋일련번호'], row['고객질문(요청)'])
    대화셋_set.add(row['대화셋일련번호'])

0 A4076 미성년 자녀도 인터넷뱅킹 가능한가요?
1 A4076 
2 A4076 준비할 서류가 뭐죠?
3 A4076 아, 학생증도 가능한가요?
4 A4076 
5 A4076 계좌 개설은 가까운 OO은행에 가서 하면 되나요>
6 A4076 
7 A4076 가까운 OO은행을 안내 받을수 있을까요?
8 A4076 
9 A4076 
10 A4079 공인인증서를 온라인에서 발급 가능한가요?
11 A4079 어떻게 해야 하나요?
12 A4079 저 지금 웹사이트 들어왔는데, 어디서 해요?
13 A4079 네. 발급 누르면 되나요?
14 A4079 혹시 핸드폰 필요한가요?
15 A4079 제 명의로 된 폰이어야 하나요?
16 A4079 제 폰이 부모님 명의로 되어 있는데.. 그럼 어떡하죠?
17 A4079 
18 A4081 안녕하세요. 공인인증서 관련해서 문의좀 드리려구요
19 A4081 은행에서 받은 공인인증서가 있는데, 증권사에서도 이용 가능할까요? 
20 A4081 그럼 증권사는 따로 발행해야 하나요?
21 A4081 범용공인인증서 받으면 은행이랑 증권사에서 둘 다 사용 가능한가요?
22 A4081 그럼 범용공인인증서도 홈페이지에서 발급가능해요?
23 A4081 발급가능시간이 있나요?
24 A4081 발급비용은 얼마죠?
25 A4081 발급에 필요한 서류가 있나요?
26 A4086 기업인터넷 뱅킹의 공인인중서 암호 오류가 납니다. 어떻게 해야 하나요?
27 A4086 기업인테넛 뱅킹의 공인인중서 암호를 잊었는데 혹시 오류횟수 제한이 있을까요?
28 A4086 3회 입력오류라고 뜨는데 어떻게 하죠?
29 A4086 인증서 암호는 몇자리 수 이상으로 정해진게 있나요?
30 A4086 어떤 형식으로 암호를 만들어야 하나요?
31 A4086 인증서 재발급 경로는 어떻게 되나요?
32 A4086 인증서 재발급시 필요한 정보들이 있을까요?
33 A4285 제가 OTP를 분실했는데요.
34 A4285 
35 A4285 
36 A4285 제가 분실한 OTP가 범죄에 사용될 수 있

In [ ]:
대화셋_set

{'A13159',
 'A18256',
 'A18470',
 'A18475',
 'A18483',
 'A18486',
 'A18507',
 'A18526',
 'A18528',
 'A18529',
 'A18539',
 'A18626',
 'A18629',
 'A18678',
 'A28443',
 'A32085',
 'A32087',
 'A32092',
 'A4076',
 'A4079',
 'A4081',
 'A4086',
 'A4285',
 'A4286',
 'A6079',
 'A7045',
 'A7052'}

In [ ]:
filtered_df = df[df['대화셋일련번호'].isin(대화셋_set)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
20,금융/보험,사고 및 보상 문의,A4076,고객,1,공인인증서,,Q,미성년 자녀도 인터넷뱅킹 가능한가요?,,,,"미성년자, 인터넷뱅킹, 공인인증서",미성년자/청소년/나이/ 인터넷뱅킹/온라인뱅킹/스마트뱅킹/ 공인인증서/전자신분증,"인터넷뱅킹,전자신분증"
21,금융/보험,사고 및 보상 문의,A4076,상담사,2,,공인인증서,Q,,자녀 명의로 OO은행계좌가 있으신가요?,,,"농협, 계좌",농협/은행,"계좌,은행"
22,금융/보험,사고 및 보상 문의,A4076,고객,3,공인인증서,,A,,,아니요.,,,,
23,금융/보험,사고 및 보상 문의,A4076,상담사,4,,공인인증서,A,,,,OO은행 계좌가 없으신 경우 계좌를 먼저 개설하셔야 하고 신분증 지참 후 방문해 주...,"농협, 계좌, 개설",농협/은행,"계좌,은행"
24,금융/보험,사고 및 보상 문의,A4076,고객,5,공인인증서,,Q,준비할 서류가 뭐죠?,,,,"어떤, 서류",,서류
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83159,금융/보험,사고 및 보상 문의,A32092,고객,16,공인인증서,,Q,인증서 재발급 경로는 어떻게 되나요?,,,,"인증서, 재발급, 경로",경로/루트,"재발급,루트"
83160,금융/보험,사고 및 보상 문의,A32092,상담사,17,,공인인증서,A,,,,"당 은행 공인인증센터 메뉴 인증서 발급, 재발급으로 들어가셔서 발급신청하시면 됩니다.","공인인증센터,은행,메뉴, 인증서발급, 재발급, 발급, 신청","공인인증센터/기관, 메뉴/메뉴, 인증서발급/메뉴, 재발급/메뉴, 발급/발부, 신청/요청","은행,요청"
83161,금융/보험,사고 및 보상 문의,A32092,고객,18,공인인증서,,Q,인증서 재발급시 필요한 정보들이 있을까요?,,,,"인증서, 재발급,정보","인증서/증명서, 정보/데이터","재발급,데이터"
83162,금융/보험,사고 및 보상 문의,A32092,상담사,19,,공인인증서,A,,,,"이용자ID,자금이체비밀번호,OTP응답번호(보안카드)가 필요합니다.","이용자ID,자금이체비밀번호,OTP응답번호,보안카드","이용자ID/아이디, 자금이체비밀번호/비밀번호, OTP응답번호/보안카드응답번호, 보안...","자금이체비밀번호,OTP"


In [ ]:
# '대화셋일련번호'로 그룹화하고 각 그룹을 인덱스 순서대로 정렬하여 원하는 컬럼들을 하나의 문자열로 합치기
def combine_columns(group):
    sorted_group = group.sort_index()  # 인덱스 순서대로 정렬
    combined_string = " ".join(
        sorted_group[['고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변']].astype(str).values.flatten()
    )

    customer_intent = sorted_group['고객의도'].iloc[0]  # 그룹의 첫 번째 '고객의도' 값 가져오기
    return pd.Series({'combined_string': combined_string, '고객의도': customer_intent})

In [ ]:
combined_result = filtered_df.groupby('대화셋일련번호').apply(combine_columns).reset_index()
combined_result

,대화셋일련번호,combined_string,고객의도
0,A13159,"안녕하세요 공인인증서 재발급을 받으려고 합니다. 네, 만료일이 언제인가요? ...",공인인증서
1,A18256,타행에서 발급받은 OTP기기를 등록하려고 하는데 잘 안됩니다. 네 혹시 저희...,타행 OTP 등록
2,A18470,스마트폰에 있는 인증서를 가지고 사용하고 싶은데요. 네 고객님. 스마트폰에 ...,스마트폰 공인인증서 복사
3,A18475,집과 회사 양쪽에서 인터넷 뱅킹이 사용 가능할까요? 네. 공인인증서를 복...,공인인증서 자택 직장병용
4,A18483,안녕하세요. 인증서에 대해서 좀 여쭙고자 합니다. 네 어떤 점이 궁금하십니까...,증권사 공인인증서
5,A18486,컴퓨터를 포맷하고 나서 공인인증서가 없어졌어요. 네. 포맷 전에 공인인증서를...,공인인증서 분실
6,A18507,일반 공인인증서랑 범용 공인인증서를 같이 사용할 수 있는지 문의드립니다. ...,일반 공인인증서와 범용 공인인증서 병용
7,A18526,OTP를 잃어버렸어요. 네 고객님. OTP를 분실하셔서 많이 불편하시겠습...,OTP 분실
8,A18528,안녕하세요. 지금 OTP를 사용중인데 이걸 제가 가지고 있는 보안카드로 변경할 수 ...,OTP 보안카드 변경
9,A18529,OTP에서 보안카드로 바꾸면 이체 한도가 줄어드나요? 네 그렇습니다. 왜...,OTP 보안카드 이체한도


In [ ]:
# '고객요청의도' 컬럼 생성
def assign_customer_request_intent(intent):
    if intent in 공인인증서_키워드_포함_고객의도_리스트:
        return '공인인증서'
    elif intent in OTP_키워드_포함_고객의도_리스트:
        return 'OTP'

combined_result['고객요청의도'] = combined_result['고객의도'].apply(assign_customer_request_intent)
combined_result

,대화셋일련번호,combined_string,고객의도,고객요청의도
0,A13159,"안녕하세요 공인인증서 재발급을 받으려고 합니다. 네, 만료일이 언제인가요? ...",공인인증서,공인인증서
1,A18256,타행에서 발급받은 OTP기기를 등록하려고 하는데 잘 안됩니다. 네 혹시 저희...,타행 OTP 등록,OTP
2,A18470,스마트폰에 있는 인증서를 가지고 사용하고 싶은데요. 네 고객님. 스마트폰에 ...,스마트폰 공인인증서 복사,공인인증서
3,A18475,집과 회사 양쪽에서 인터넷 뱅킹이 사용 가능할까요? 네. 공인인증서를 복...,공인인증서 자택 직장병용,공인인증서
4,A18483,안녕하세요. 인증서에 대해서 좀 여쭙고자 합니다. 네 어떤 점이 궁금하십니까...,증권사 공인인증서,공인인증서
5,A18486,컴퓨터를 포맷하고 나서 공인인증서가 없어졌어요. 네. 포맷 전에 공인인증서를...,공인인증서 분실,공인인증서
6,A18507,일반 공인인증서랑 범용 공인인증서를 같이 사용할 수 있는지 문의드립니다. ...,일반 공인인증서와 범용 공인인증서 병용,공인인증서
7,A18526,OTP를 잃어버렸어요. 네 고객님. OTP를 분실하셔서 많이 불편하시겠습...,OTP 분실,OTP
8,A18528,안녕하세요. 지금 OTP를 사용중인데 이걸 제가 가지고 있는 보안카드로 변경할 수 ...,OTP 보안카드 변경,OTP
9,A18529,OTP에서 보안카드로 바꾸면 이체 한도가 줄어드나요? 네 그렇습니다. 왜...,OTP 보안카드 이체한도,OTP


In [ ]:
combined_result[combined_result['대화셋일련번호']=='A28443']['combined_string'].values[0]

'안녕하세요 S은행이죠?       네 고객님 말씀하세요 다른게 아니구요 제가 지금 지갑을 잃어버려서요 OPT카드로 없어졌거든요?       네 많이 당황하셨겠습니다    우선 고객님 인적사항 부터 확인후 안내도와드리겠습니다  성함이요?     OOO   주민번호앞자리 말씀해주세요?     OOOOO입니다     네 잠시만 기다려주세요    네     00입출금 통장과 연계된 opt 맞으시죠   네  재발급 받으려는데 어떻게 해야 하나요?       본인이 직접 오실경우 신분증을 지참하시어 가까운 영업점을 방문하시면 됩니다 혹시 스마트뱅킹에서도 가능할까요?       네 모바일OTP도 있습니다. 그건 스마트뱅킹을 통해 발급 가능합니다   네 감사합니다 '

In [ ]:
# 각 row의 값들을 순회하면서 출력
for index, row in combined_result.iterrows():
    print(f"대화셋일련번호: {row['대화셋일련번호']}, 고객요청의도: {row['고객요청의도']},  Combined String: {row['combined_string']}")

대화셋일련번호: A13159, 고객요청의도: 공인인증서,  Combined String: 안녕하세요 공인인증서 재발급을 받으려고 합니다.     네, 만료일이 언제인가요?   10월 15일입니다. 연장 가능할까요?       네, 만료일로부터 30일 전부터 가능합니다. 은행에 가서 하면되는거죠?       네, 맞습니다.  뭐 가져갈 거 있나요?       신분증 지참하셔야 합니다.  연장하면 얼마나 되는거에요?       1년입니다.  1년 이상은 안될까요?       죄송하지만, 1년만 가능합니다.  네, 혹시 온라인으로도 연장 가능한가요?       은행 공인인증센터에 접속하시면 공인인증서 갱신하실 수 있습니다.  네, 만약 만료일이 지나면, 갱신이 불가능한가요?       네 그렇습니다. 그럴 경우, 인증서 재발급 진행을 해주셔야 합니다.    네, 알겠습니다.    혹시 더 궁금한 사항 있으신가요?     아니요. 수고하세요.      네. 감사합니다. 
대화셋일련번호: A18256, 고객요청의도: OTP,  Combined String: 타행에서 발급받은 OTP기기를 등록하려고 하는데 잘 안됩니다.     네 혹시 저희 은행에 최초로 등록하시는 건가요?     아니오. 발급은 다른 데서 받았고 여기 은행에서 이미 등록해서 사용하고 있었습니다.   그럼 이번에 OTP기기를 재발급 받으신건가요?     네.   최초 발급받으신 곳에서 재발급 받으셨나요?     네. 기기가 고장나서 새로 받았어요.     불량으로 인한 재발급이시면 저희 은행 인터넷뱅킹에서 등록하실 수 있습니다. 어디서 하나요?       인터넷 뱅킹에 로그인하셔서 전체서비스 -> 보안센터 -> OTP발생기 -> 불량교체 타기관 OTP발생기 이용등록 에서 가능하십니다. 등록하려는데 안되네요. 어떻게 해야 하나요?     네. 혹시 어떤 부분에서 막히시나요?     법인고객이라 지점으로 문의하라고 나옵니다.     법인, 기업 고객인 경우 '온라인 상 자동등록대상'인 경우에 한해 인터넷 뱅

# Document 생성 및 Vectorstore에 Embedding해서 저장

In [ ]:
from langchain.schema import Document

docs = []
공인인증서_docs = []
OTP_docs = []

# 각 row의 값들을 순회하면서 출력
for index, row in combined_result.iterrows():

    doc = Document(page_content=row['combined_string'], metadata={'대화셋일련번호': row['대화셋일련번호'], '고객요청의도': row['고객요청의도']})
    docs.append(doc)

    if row['고객요청의도'] == '공인인증서':
        공인인증서_docs.append(doc)
    elif row['고객요청의도'] == 'OTP':
        OTP_docs.append(doc)

In [ ]:
docs

[Document(page_content='안녕하세요 공인인증서 재발급을 받으려고 합니다.     네, 만료일이 언제인가요?   10월 15일입니다. 연장 가능할까요?       네, 만료일로부터 30일 전부터 가능합니다. 은행에 가서 하면되는거죠?       네, 맞습니다.  뭐 가져갈 거 있나요?       신분증 지참하셔야 합니다.  연장하면 얼마나 되는거에요?       1년입니다.  1년 이상은 안될까요?       죄송하지만, 1년만 가능합니다.  네, 혹시 온라인으로도 연장 가능한가요?       은행 공인인증센터에 접속하시면 공인인증서 갱신하실 수 있습니다.  네, 만약 만료일이 지나면, 갱신이 불가능한가요?       네 그렇습니다. 그럴 경우, 인증서 재발급 진행을 해주셔야 합니다.    네, 알겠습니다.    혹시 더 궁금한 사항 있으신가요?     아니요. 수고하세요.      네. 감사합니다. ', metadata={'대화셋일련번호': 'A13159', '고객요청의도': '공인인증서'}),
 Document(page_content="타행에서 발급받은 OTP기기를 등록하려고 하는데 잘 안됩니다.     네 혹시 저희 은행에 최초로 등록하시는 건가요?     아니오. 발급은 다른 데서 받았고 여기 은행에서 이미 등록해서 사용하고 있었습니다.   그럼 이번에 OTP기기를 재발급 받으신건가요?     네.   최초 발급받으신 곳에서 재발급 받으셨나요?     네. 기기가 고장나서 새로 받았어요.     불량으로 인한 재발급이시면 저희 은행 인터넷뱅킹에서 등록하실 수 있습니다. 어디서 하나요?       인터넷 뱅킹에 로그인하셔서 전체서비스 -> 보안센터 -> OTP발생기 -> 불량교체 타기관 OTP발생기 이용등록 에서 가능하십니다. 등록하려는데 안되네요. 어떻게 해야 하나요?     네. 혹시 어떤 부분에서 막히시나요?     법인고객이라 지점으로 문의하라고 나옵니다.     법인, 기업 고객인 경우 '온라인 상 자동등록대상'인 경우에 한해 인

In [ ]:
len(docs)

27

In [ ]:
공인인증서_docs

[Document(page_content='안녕하세요 공인인증서 재발급을 받으려고 합니다.     네, 만료일이 언제인가요?   10월 15일입니다. 연장 가능할까요?       네, 만료일로부터 30일 전부터 가능합니다. 은행에 가서 하면되는거죠?       네, 맞습니다.  뭐 가져갈 거 있나요?       신분증 지참하셔야 합니다.  연장하면 얼마나 되는거에요?       1년입니다.  1년 이상은 안될까요?       죄송하지만, 1년만 가능합니다.  네, 혹시 온라인으로도 연장 가능한가요?       은행 공인인증센터에 접속하시면 공인인증서 갱신하실 수 있습니다.  네, 만약 만료일이 지나면, 갱신이 불가능한가요?       네 그렇습니다. 그럴 경우, 인증서 재발급 진행을 해주셔야 합니다.    네, 알겠습니다.    혹시 더 궁금한 사항 있으신가요?     아니요. 수고하세요.      네. 감사합니다. ', metadata={'대화셋일련번호': 'A13159', '고객요청의도': '공인인증서'}),
 Document(page_content='스마트폰에 있는 인증서를 가지고 사용하고 싶은데요.     네 고객님. 스마트폰에 있는 인증서를 활용하고 싶으신 건가요?     네.   어디에 사용하고 싶으신 건가요?     연말정산 홈페이지에서 사용하고 싶어요.   연말정산 홈페이지는 PC에서 이용하실 계획이십니까?     네.     그러시면 스마트폰 속의 인증서를 PC로 옮겨주셔야 합니다. 어떻게요?    지금 PC 옆에 계신가요?    아뇨. 컴퓨터 옆으로 갈게요 잠깐만 기다려주시겠어요?       네. 네 이제 설명해주시겠어요?     네. 스마트폰에서 앱에 들어가보시겠습니까?   네 들어왔어요. 다음은요?       인증/보안 메뉴에서 인증서 복사에 들어가주십시오.   네.     휴대폰->PC 복사를 클릭해주시면 됩니다.   네 복사됐어요.     이제 연말정산 홈페이지에서 공인인증서를 등록해서 사용하시면 됩니다.', metadata={'대화셋

In [ ]:
len(공인인증서_docs)

17

In [ ]:
OTP_docs

[Document(page_content="타행에서 발급받은 OTP기기를 등록하려고 하는데 잘 안됩니다.     네 혹시 저희 은행에 최초로 등록하시는 건가요?     아니오. 발급은 다른 데서 받았고 여기 은행에서 이미 등록해서 사용하고 있었습니다.   그럼 이번에 OTP기기를 재발급 받으신건가요?     네.   최초 발급받으신 곳에서 재발급 받으셨나요?     네. 기기가 고장나서 새로 받았어요.     불량으로 인한 재발급이시면 저희 은행 인터넷뱅킹에서 등록하실 수 있습니다. 어디서 하나요?       인터넷 뱅킹에 로그인하셔서 전체서비스 -> 보안센터 -> OTP발생기 -> 불량교체 타기관 OTP발생기 이용등록 에서 가능하십니다. 등록하려는데 안되네요. 어떻게 해야 하나요?     네. 혹시 어떤 부분에서 막히시나요?     법인고객이라 지점으로 문의하라고 나옵니다.     법인, 기업 고객인 경우 '온라인 상 자동등록대상'인 경우에 한해 인터넷 뱅킹에서 등록이 가능합니다. 그 대상이 아닌 경우는 어떻게 하죠?       온라인 상 자동등록대상'이 아니실 경우 영업점에 방문하여 주시기 바랍니다. 영업점에 꼭 가서 해야만 하나요?       네 그렇습니다. 뭐뭐 챙겨가야 하나요?       필요서류와 타기관 OTP 발생기를 지참하여 주십시오.", metadata={'대화셋일련번호': 'A18256', '고객요청의도': 'OTP'}),
 Document(page_content='OTP를 잃어버렸어요.       네 고객님. OTP를 분실하셔서 많이 불편하시겠습니다.  OTP분실신고부터 접수해드릴까요?     네.   고객님 성함이 OOO 맞으십니까?     네 맞습니다.   생년월일 OOOOOOO 맞으십니까?     네.   휴대폰 끝번호 4자리 불러주시겠습니까?     0000이요.     네 확인되셨습니다.    OTP 발생기는 분실신고 접수해 드렸습니다. 제가 여러 은행을 이 한 OTP로 이용중이었는데 그럼 이제 모든 은행에서 다 이용이 안되는거죠?

In [ ]:
len(OTP_docs)

10

# Embedding

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sroberta-multitask" # (KorNLU 데이터셋에 학습시킨 한국어 임베딩 모델)
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_model, collection_name="all")

In [ ]:
공인인증서_vectorstore = Chroma.from_documents(documents=공인인증서_docs, embedding=embedding_model, collection_name="certificate")

In [ ]:
OTP_vectorstore = Chroma.from_documents(documents=OTP_docs, embedding=embedding_model, collection_name="OTP")

In [ ]:
len(vectorstore)

27

In [ ]:
len(공인인증서_vectorstore)

17

In [ ]:
len(OTP_vectorstore)

10

# Sanity Check

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
OTP_retriever = OTP_vectorstore.as_retriever(search_kwargs={"k": 1})
공인인증서_retriever = 공인인증서_vectorstore.as_retriever(search_kwargs={"k": 1})

In [ ]:
retrieved_docs = retriever.invoke(
    "OTP를 분실했는데 어떻게 하나요?"
)
print(retrieved_docs[0].page_content)

OTP를 잃어버렸어요.       네 고객님. OTP를 분실하셔서 많이 불편하시겠습니다.  OTP분실신고부터 접수해드릴까요?     네.   고객님 성함이 OOO 맞으십니까?     네 맞습니다.   생년월일 OOOOOOO 맞으십니까?     네.   휴대폰 끝번호 4자리 불러주시겠습니까?     0000이요.     네 확인되셨습니다.    OTP 발생기는 분실신고 접수해 드렸습니다. 제가 여러 은행을 이 한 OTP로 이용중이었는데 그럼 이제 모든 은행에서 다 이용이 안되는거죠?       네 그렇습니다.    OTP는 전 금융기관 통합으로 사용하며 분실신고 즉시 정보가 모두 공유되어 어느 은행에서도 사용할 수 없습니다.  영업시간내에 지점으로 방문하셔서 재발급 받으시겠습니까?   그렇군요. 일때문에 바빠서 방문이 어려운데 혹시 모바일로 이용하는 OTP같은 건 없나요?     네 고객님. 있습니다. 디지털 OTP가 있는데 안내해드릴까요?   네 안내해주실래요?       네. 해당부서 연결해드리겠습니다. 잠시만 기다려주십시오.


In [ ]:
retrieved_docs = OTP_retriever.invoke(
    "OTP를 분실했는데 어떻게 하나요?"
)
print(retrieved_docs[0].page_content)

OTP를 잃어버렸어요.       네 고객님. OTP를 분실하셔서 많이 불편하시겠습니다.  OTP분실신고부터 접수해드릴까요?     네.   고객님 성함이 OOO 맞으십니까?     네 맞습니다.   생년월일 OOOOOOO 맞으십니까?     네.   휴대폰 끝번호 4자리 불러주시겠습니까?     0000이요.     네 확인되셨습니다.    OTP 발생기는 분실신고 접수해 드렸습니다. 제가 여러 은행을 이 한 OTP로 이용중이었는데 그럼 이제 모든 은행에서 다 이용이 안되는거죠?       네 그렇습니다.    OTP는 전 금융기관 통합으로 사용하며 분실신고 즉시 정보가 모두 공유되어 어느 은행에서도 사용할 수 없습니다.  영업시간내에 지점으로 방문하셔서 재발급 받으시겠습니까?   그렇군요. 일때문에 바빠서 방문이 어려운데 혹시 모바일로 이용하는 OTP같은 건 없나요?     네 고객님. 있습니다. 디지털 OTP가 있는데 안내해드릴까요?   네 안내해주실래요?       네. 해당부서 연결해드리겠습니다. 잠시만 기다려주십시오.


In [ ]:
retrieved_docs = 공인인증서_retriever.invoke(
    "OTP를 분실했는데 어떻게 하나요?"
)
print(retrieved_docs[0].page_content)

공인인증서를 분실했어요.     네 고객님. 공인인증서를 분실하셨다구요?     USB에 담아놨는데 없어졌어요.     네 그럼 재발급을 받으셔야 합니다. 어떻게 받죠?       영업점으로 방문하시면 자세히 안내드리겠습니다. 뭐뭐 챙겨가야 하나요?       신분증만 있으시면 됩니다. 네. 가까운 영업점 좀 안내해주실래요?     고객님이 위치하신 지역이 어디신가요?     OO시 OO구 OO동입니다.   네. OO동에 지점이 하나 있습니다. 주소 알려드릴까요?   네 알려주시겠어요?       OO동 OOO번지입니다. 네 감사합니다. 혹시 수수료가 있을까요?       공인인증서 재발급은 수수료가 없습니다. OTP는 그대로 사용할 수 있나요?       네 그대로 사용가능합니다. 혹시 영업시간은 언제까지이죠?       오후 네시까지 방문해주시면 됩니다.


In [ ]:
retrieved_docs = retriever.invoke(
    "OTP를 재발급 받으려면 어떻게해야 하나요?"
)
print(retrieved_docs[0].page_content)

안녕하세요. 지금 OTP를 사용중인데 이걸 제가 가지고 있는 보안카드로 변경할 수 있나요?     네 고객님. 사용중인 OTP를 보안카드로 변경하신다는 말씀이십니까?     네.   OTP가 사용에 있어 더 편리하실텐데요, 변경하시려는 이유가 뭔지 여쭈어봐도 됩니까?     네. OTP 배터리가 다 되었는데 교체하러 갈 시간이 없어요.     아 그러시군요. 그러시면 몇가지 확인을 거친 후 안내 드리겠습니다.   네.   지금 현재 보안카드를 가지고 계십니까?     네 예전에 받았던 게 있어요.   OTP나 보안카드 둘 다 사고등록이 없어야 합니다. 맞으십니까?   사고등록이란 분실을 말씀하시는 건가요?     네 맞습니다. 분실신고 한 적 있으십니까?     없어요.   고객정보로 등록된 전화번호로 2채널 인증등 추가 인증이 가능하셔야 합니다. 동의하십니까?     네.   현재 공인인증서가 있으십니까?     네 있어요.     네 그럼 로그인→ 전체메뉴→ 인증/보안→ 보안카드/OTP→ 보안수단변경으로 가셔서 변경하실 수 있습니다. 그럼 바로 사용 가능한가요?       네. 단 보안카드로 변경되면 보안등급이 하향조정되어 이체한도가 축소되는 점에 유의하시기 바랍니다.


In [ ]:
retrieved_docs = OTP_retriever.invoke(
    "OTP를 재발급 받으려면 어떻게해야 하나요?"
)
print(retrieved_docs[0].page_content)

안녕하세요. 지금 OTP를 사용중인데 이걸 제가 가지고 있는 보안카드로 변경할 수 있나요?     네 고객님. 사용중인 OTP를 보안카드로 변경하신다는 말씀이십니까?     네.   OTP가 사용에 있어 더 편리하실텐데요, 변경하시려는 이유가 뭔지 여쭈어봐도 됩니까?     네. OTP 배터리가 다 되었는데 교체하러 갈 시간이 없어요.     아 그러시군요. 그러시면 몇가지 확인을 거친 후 안내 드리겠습니다.   네.   지금 현재 보안카드를 가지고 계십니까?     네 예전에 받았던 게 있어요.   OTP나 보안카드 둘 다 사고등록이 없어야 합니다. 맞으십니까?   사고등록이란 분실을 말씀하시는 건가요?     네 맞습니다. 분실신고 한 적 있으십니까?     없어요.   고객정보로 등록된 전화번호로 2채널 인증등 추가 인증이 가능하셔야 합니다. 동의하십니까?     네.   현재 공인인증서가 있으십니까?     네 있어요.     네 그럼 로그인→ 전체메뉴→ 인증/보안→ 보안카드/OTP→ 보안수단변경으로 가셔서 변경하실 수 있습니다. 그럼 바로 사용 가능한가요?       네. 단 보안카드로 변경되면 보안등급이 하향조정되어 이체한도가 축소되는 점에 유의하시기 바랍니다.


In [ ]:
retrieved_docs = 공인인증서_retriever.invoke(
    "OTP를 재발급 받으려면 어떻게해야 하나요?"
)
print(retrieved_docs[0].page_content)

공인인증서를 분실했어요.     네 고객님. 공인인증서를 분실하셨다구요?     USB에 담아놨는데 없어졌어요.     네 그럼 재발급을 받으셔야 합니다. 어떻게 받죠?       영업점으로 방문하시면 자세히 안내드리겠습니다. 뭐뭐 챙겨가야 하나요?       신분증만 있으시면 됩니다. 네. 가까운 영업점 좀 안내해주실래요?     고객님이 위치하신 지역이 어디신가요?     OO시 OO구 OO동입니다.   네. OO동에 지점이 하나 있습니다. 주소 알려드릴까요?   네 알려주시겠어요?       OO동 OOO번지입니다. 네 감사합니다. 혹시 수수료가 있을까요?       공인인증서 재발급은 수수료가 없습니다. OTP는 그대로 사용할 수 있나요?       네 그대로 사용가능합니다. 혹시 영업시간은 언제까지이죠?       오후 네시까지 방문해주시면 됩니다.


# Multiple Retriever를 활용하여 연관된 문서 찾아오기

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_KEY)

In [ ]:
retrievers = {
    "": retriever,
    "공인인증서": 공인인증서_retriever,
    "OTP": OTP_retriever,
}

In [ ]:
from langchain_core.runnables import chain

In [ ]:
@chain
def custom_chain(question):
    response = extraction_chain.run(question)
    #print(response.get('고객요청의도', ""))
    retriever = retrievers[response.get('고객요청의도') or ""]
    return retriever.invoke(question)

In [ ]:
custom_chain.invoke("공인인증서 폐지는 언제부터 실시되나요?")

[Document(page_content='공인인증서 폐지는 언제부터 실시되나요?       2020년 12월부터입니다. 공인인증서가 폐지되면 이제 공인인증서는 사용 못하는 건가요?       그렇지 않습니다. 공인인증서 의무사용이 폐지된 것으로 원하는 이용자들은 기존의 공인인증서를 계속해서 사용하실 수 있습니다. 굳이 공인인증서를 폐지하는 이유는 뭔가요?       공인인증서를 사용하기 위해 설치하는 액티브X컨트롤가 무겁고 불편하며 비효율적이라는 지적이 계속 있어왔습니다. 또 다른 이유도 있나요?       공인인증서는 반드시 대면발급해야 하므로 해외 이용자들이 이용하기가 매우 번거롭기 때문입니다. 앞으로는 어떤식으로 인증하게 되는 건가요?       앞으로 금융기관들이 독자적으로 개발한 보안플랫폼들이 활발히 이용될 것으로 예상됩니다. 어떤 보안들이 있을까요?       생체보안, 전자서명 등이 대두되고 있습니다. 생체보안은 지문인식을 말하나요?       지문, 홍채, 안면인식, 혈관인식 등 다양한 방법들이 시도되고 있습니다. 전자서명은 어떤건가요?       모바일인증기 등을 이용한 전자로 서명하는 방식을 말합니다. 전자서명의 장점은 뭐가 있을까요?       분산신원확인 DID서비스가 가능합니다. 그게 뭔가요?       한번 인증하면 다양한 기관서비스를 모두 이용할 수 있는 것입니다.', metadata={'고객요청의도': '공인인증서', '대화셋일련번호': 'A18626'})]

In [ ]:
custom_chain.invoke("제 OTP가 에러가 났는데 어떻게 하죠?")

[Document(page_content='제 OTP가 에러가 났는데 어떻게 하죠?     어떻게 에러가 났나요?     OTP버튼을 눌렀는데 숫자가 안나오고 영어가 나와요.   어떻게 표시되나요?     dnLd 13이라고 뜨고 깜빡이다가 꺼져요.     배터리가 없거나 배터리 손상으로 사용이 불가능한 상태입니다. 저는 OTP교체한지 얼마 안되었는데요?       가끔 불량제품이 발생될 수 있습니다. 지점에 방문해서 교체해주세요. 우편으로 발송해줄수는 없나요?       죄송합니다만 본인확인 후 전달해드려야 하므로 지점에 방문하여 주시기 바랍니다. 비용이 발생하나요?       보증기한 내에는 무료이고 그 이후에는 00원이 발생됩니다. 보증기한을 얼마나 되죠?       0년입니다. 그럼 저는 0월에 교체했으니 무료로 되겠네요?       네 무료입니다. 은행 방문할때 어떤 서류가 필요한가요?       신분증만 지참하셔서 방문하시면 됩니다. 혹시 아내가 대신 가도 되나요?       대리인이 방문하실때는 위임장과 대리인의 신분증이 추가로 필요합니다.', metadata={'고객요청의도': 'OTP', '대화셋일련번호': 'A18678'})]

In [ ]:
custom_chain.invoke("안녕하세요 S은행이죠?")

[Document(page_content='안녕하세요 S은행이죠?       네 고객님 말씀하세요 다른게 아니구요 제가 지금 지갑을 잃어버려서요 OPT카드로 없어졌거든요?       네 많이 당황하셨겠습니다    우선 고객님 인적사항 부터 확인후 안내도와드리겠습니다  성함이요?     OOO   주민번호앞자리 말씀해주세요?     OOOOO입니다     네 잠시만 기다려주세요    네     00입출금 통장과 연계된 opt 맞으시죠   네  재발급 받으려는데 어떻게 해야 하나요?       본인이 직접 오실경우 신분증을 지참하시어 가까운 영업점을 방문하시면 됩니다 혹시 스마트뱅킹에서도 가능할까요?       네 모바일OTP도 있습니다. 그건 스마트뱅킹을 통해 발급 가능합니다   네 감사합니다 ', metadata={'고객요청의도': 'OTP', '대화셋일련번호': 'A28443'})]

# 대화 history 저장을 위한 ConversationBufferMemory 설정

**ConversationBufferMemory** : 모든 대화내역을 저장

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

In [ ]:
memory.load_memory_variables({})

{'history': ''}

In [ ]:
def load_memory(input):
    return memory.load_memory_variables({})["history"]

# Tracking을 위한 LangSmith 설정(Optional)

In [ ]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"User Intent Analysis - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "여러분의_LangSmith_API_key"

In [ ]:
unique_id

'1234'

In [ ]:
memory.load_memory_variables({})["history"]

''

In [ ]:
def using_intent_get_relevant_document(user_question):
    histroy_combined_question =  memory.load_memory_variables({})["history"] + '\nHuman::' + user_question

    relevant_document = custom_chain.invoke(histroy_combined_question)[0]

    # print(histroy_combined_question)
    # print(relevant_document)
    return relevant_document

In [ ]:
relevant_document = using_intent_get_relevant_document("공인인증서 폐지는 언제부터 실시되나요?")
relevant_document

Document(page_content='공인인증서 폐지는 언제부터 실시되나요?       2020년 12월부터입니다. 공인인증서가 폐지되면 이제 공인인증서는 사용 못하는 건가요?       그렇지 않습니다. 공인인증서 의무사용이 폐지된 것으로 원하는 이용자들은 기존의 공인인증서를 계속해서 사용하실 수 있습니다. 굳이 공인인증서를 폐지하는 이유는 뭔가요?       공인인증서를 사용하기 위해 설치하는 액티브X컨트롤가 무겁고 불편하며 비효율적이라는 지적이 계속 있어왔습니다. 또 다른 이유도 있나요?       공인인증서는 반드시 대면발급해야 하므로 해외 이용자들이 이용하기가 매우 번거롭기 때문입니다. 앞으로는 어떤식으로 인증하게 되는 건가요?       앞으로 금융기관들이 독자적으로 개발한 보안플랫폼들이 활발히 이용될 것으로 예상됩니다. 어떤 보안들이 있을까요?       생체보안, 전자서명 등이 대두되고 있습니다. 생체보안은 지문인식을 말하나요?       지문, 홍채, 안면인식, 혈관인식 등 다양한 방법들이 시도되고 있습니다. 전자서명은 어떤건가요?       모바일인증기 등을 이용한 전자로 서명하는 방식을 말합니다. 전자서명의 장점은 뭐가 있을까요?       분산신원확인 DID서비스가 가능합니다. 그게 뭔가요?       한번 인증하면 다양한 기관서비스를 모두 이용할 수 있는 것입니다.', metadata={'고객요청의도': '공인인증서', '대화셋일련번호': 'A18626'})

# 최종 Prompt 및 Chain 생성

In [ ]:
from langchain.prompts import PromptTemplate

template = """다음과 같은 맥락과 채팅히스토리를 사용하여 마지막 질문에 대답하십시오.
맥락: {context}
채팅히스토리 : {history}
질문: {question}
도움이 되는 답변:"""
rag_prompt_custom = PromptTemplate.from_template(template)

In [ ]:
# RAG chain 설정
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder

rag_chain = {"context": using_intent_get_relevant_document, "question": RunnablePassthrough(), "history": load_memory} | rag_prompt_custom | llm

In [ ]:
def get_result_and_save_memory(user_question):
  chat_result = rag_chain.invoke(user_question)

  # memory에 저장
  memory.chat_memory.add_user_message(user_question)
  memory.chat_memory.add_ai_message(chat_result.content)

  return chat_result.content

# OTP, 공인인증서 AI 고객센터 챗봇 성능 evaluation (정성적 평가)

## Case 1 - OTP


In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A18526']
temp_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
46228,금융/보험,사고 및 보상 문의,A18526,고객,1,OTP 분실,,Q,OTP를 잃어버렸어요.,,,,OTP,OTP/보안,"OTP,보안"
46229,금융/보험,사고 및 보상 문의,A18526,상담사,2,,OTP 분실,A,,,,네 고객님. OTP를 분실하셔서 많이 불편하시겠습니다.,"OTP, 분실","OTP/보안, 분실/사고","분실,사고"
46230,금융/보험,사고 및 보상 문의,A18526,상담사,3,,OTP 분실,Q,,OTP분실신고부터 접수해드릴까요?,,,"분실신고, OTP","분실신고/사고접수, OTP/보안","OTP,보안"
46231,금융/보험,사고 및 보상 문의,A18526,고객,4,OTP 분실,,A,,,네.,,접수,접수/등록,"접수,등록"
46232,금융/보험,사고 및 보상 문의,A18526,상담사,5,,OTP 분실,Q,,고객님 성함이 OOO 맞으십니까?,,,성함,성함/개인정보,"성함,개인정보"
46233,금융/보험,사고 및 보상 문의,A18526,고객,6,OTP 분실,,A,,,네 맞습니다.,,,,
46234,금융/보험,사고 및 보상 문의,A18526,상담사,7,,OTP 분실,Q,,생년월일 OOOOOOO 맞으십니까?,,,생년월일,"날짜, 생년월일/개인정보","생년월일,개인정보"
46235,금융/보험,사고 및 보상 문의,A18526,고객,8,OTP 분실,,A,,,네.,,,,
46236,금융/보험,사고 및 보상 문의,A18526,상담사,9,,OTP 분실,Q,,휴대폰 끝번호 4자리 불러주시겠습니까?,,,휴대폰,휴대폰/스마트폰,"휴대폰,스마트폰"
46237,금융/보험,사고 및 보상 문의,A18526,고객,10,OTP 분실,,A,,,0000이요.,,,,


In [ ]:
llm_response = get_result_and_save_memory("OTP를 잃어버렸어요.")
llm_response

'네 고객님, OTP를 분실하셔서 많이 불편하시겠습니다. OTP 분실신고부터 접수해드릴까요?'

In [ ]:
memory.load_memory_variables({})["history"]

'Human: OTP를 잃어버렸어요.\nAI: 네 고객님, OTP를 분실하셔서 많이 불편하시겠습니다. OTP 분실신고부터 접수해드릴까요?'

In [ ]:
llm_response = get_result_and_save_memory("제가 여러 은행을 이 한 OTP로 이용중이었는데 분실신고하면 이제 모든 은행에서 다 이용이 안되는거죠?")
llm_response

'네, 그렇습니다. OTP는 전 금융기관 통합으로 사용되기 때문에 분실신고 즉시 정보가 모든 은행에 공유되어 어느 은행에서도 사용할 수 없게 됩니다. 영업시간 내에 가까운 지점으로 방문하셔서 재발급 받으시거나, 모바일로 이용 가능한 디지털 OTP를 이용하실 수도 있습니다. 디지털 OTP에 대한 안내가 필요하시면 말씀해 주세요.'

In [ ]:
memory.load_memory_variables({})["history"]

'Human: OTP를 잃어버렸어요.\nAI: 네 고객님, OTP를 분실하셔서 많이 불편하시겠습니다. OTP 분실신고부터 접수해드릴까요?\nHuman: 제가 여러 은행을 이 한 OTP로 이용중이었는데 분실신고하면 이제 모든 은행에서 다 이용이 안되는거죠?\nAI: 네, 그렇습니다. OTP는 전 금융기관 통합으로 사용되기 때문에 분실신고 즉시 정보가 모든 은행에 공유되어 어느 은행에서도 사용할 수 없게 됩니다. 영업시간 내에 가까운 지점으로 방문하셔서 재발급 받으시거나, 모바일로 이용 가능한 디지털 OTP를 이용하실 수도 있습니다. 디지털 OTP에 대한 안내가 필요하시면 말씀해 주세요.'

In [ ]:
llm_response = get_result_and_save_memory("그렇군요. 일때문에 바빠서 방문이 어려운데 혹시 모바일로 이용하는 OTP같은 건 없나요?")
llm_response

'네, 고객님. 모바일로 이용할 수 있는 디지털 OTP가 있습니다. 디지털 OTP는 스마트폰 앱을 통해 발급받아 사용할 수 있으며, 은행 지점을 방문하지 않고도 편리하게 이용할 수 있습니다. 디지털 OTP에 대한 자세한 안내를 원하시면 해당 부서로 연결해드리겠습니다. 잠시만 기다려주십시오.'

In [ ]:
memory.load_memory_variables({})["history"]

'Human: OTP를 잃어버렸어요.\nAI: 네 고객님, OTP를 분실하셔서 많이 불편하시겠습니다. OTP 분실신고부터 접수해드릴까요?\nHuman: 제가 여러 은행을 이 한 OTP로 이용중이었는데 분실신고하면 이제 모든 은행에서 다 이용이 안되는거죠?\nAI: 네, 그렇습니다. OTP는 전 금융기관 통합으로 사용되기 때문에 분실신고 즉시 정보가 모든 은행에 공유되어 어느 은행에서도 사용할 수 없게 됩니다. 영업시간 내에 가까운 지점으로 방문하셔서 재발급 받으시거나, 모바일로 이용 가능한 디지털 OTP를 이용하실 수도 있습니다. 디지털 OTP에 대한 안내가 필요하시면 말씀해 주세요.\nHuman: 그렇군요. 일때문에 바빠서 방문이 어려운데 혹시 모바일로 이용하는 OTP같은 건 없나요?\nAI: 네, 고객님. 모바일로 이용할 수 있는 디지털 OTP가 있습니다. 디지털 OTP는 스마트폰 앱을 통해 발급받아 사용할 수 있으며, 은행 지점을 방문하지 않고도 편리하게 이용할 수 있습니다. 디지털 OTP에 대한 자세한 안내를 원하시면 해당 부서로 연결해드리겠습니다. 잠시만 기다려주십시오.'

## Case 2 - 공인인증서

In [ ]:
# 대화내역을 저장한 Memory 초기화하기
memory.clear()

In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A4076']
temp_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
20,금융/보험,사고 및 보상 문의,A4076,고객,1,공인인증서,,Q,미성년 자녀도 인터넷뱅킹 가능한가요?,,,,"미성년자, 인터넷뱅킹, 공인인증서",미성년자/청소년/나이/ 인터넷뱅킹/온라인뱅킹/스마트뱅킹/ 공인인증서/전자신분증,"인터넷뱅킹,전자신분증"
21,금융/보험,사고 및 보상 문의,A4076,상담사,2,,공인인증서,Q,,자녀 명의로 OO은행계좌가 있으신가요?,,,"농협, 계좌",농협/은행,"계좌,은행"
22,금융/보험,사고 및 보상 문의,A4076,고객,3,공인인증서,,A,,,아니요.,,,,
23,금융/보험,사고 및 보상 문의,A4076,상담사,4,,공인인증서,A,,,,OO은행 계좌가 없으신 경우 계좌를 먼저 개설하셔야 하고 신분증 지참 후 방문해 주...,"농협, 계좌, 개설",농협/은행,"계좌,은행"
24,금융/보험,사고 및 보상 문의,A4076,고객,5,공인인증서,,Q,준비할 서류가 뭐죠?,,,,"어떤, 서류",,서류
25,금융/보험,사고 및 보상 문의,A4076,상담사,6,,공인인증서,A,,,,주민번호와 생년월일이 포함된 신분증이 필요합니다. 추가적으로 계좌 개설에 필요한 서...,"주민번호, 생년월일, 신분증, 추가적, 계좌, 개설, 서류, 발생",생년월일/나이,"생년월일,나이"
26,금융/보험,사고 및 보상 문의,A4076,고객,7,공인인증서,,Q,"아, 학생증도 가능한가요?",,,,학생증,학생증/신분증,"학생증,신분증"
27,금융/보험,사고 및 보상 문의,A4076,상담사,8,,공인인증서,Q,,학생증에 주민번호가 나와있나요?,,,"학생증, 주민번호",학생증/신분증/ 주민번호/주민등록번호,"주민번호,주민등록번호"
28,금융/보험,사고 및 보상 문의,A4076,고객,9,공인인증서,,A,,,생년월일만 나와있어요.,,생년월일,생년월일/나이,"생년월일,나이"
29,금융/보험,사고 및 보상 문의,A4076,상담사,10,,공인인증서,A,,,,그럼 학생증과 함께 주민등록번호가 표기된 주민등록초본 등 추가 지참 서류를 가져오시...,"학생증, 주민번호, 표기, 서류",학생증/신분증/ 주민번호/주민등록번호,"주민번호,주민등록번호"


In [ ]:
llm_response = get_result_and_save_memory("미성년 자녀도 인터넷뱅킹 가능한가요?	")
llm_response

'미성년 자녀도 인터넷뱅킹이 가능합니다. 다만, 자녀 명의로 OO은행 계좌가 있어야 하며, 계좌가 없는 경우 먼저 계좌를 개설해야 합니다. 계좌 개설 시에는 주민번호와 생년월일이 포함된 신분증이 필요하며, 학생증만 있는 경우 주민등록초본 등 추가 서류가 필요할 수 있습니다. 계좌 개설 후 인터넷뱅킹 신청을 진행하시면 됩니다. 가까운 OO은행 지점을 방문하여 필요한 서류를 확인하고 진행하시면 됩니다.'

In [ ]:
memory.load_memory_variables({})["history"]

'Human: 미성년 자녀도 인터넷뱅킹 가능한가요?\t\nAI: 미성년 자녀도 인터넷뱅킹이 가능합니다. 다만, 자녀 명의로 OO은행 계좌가 있어야 하며, 계좌가 없는 경우 먼저 계좌를 개설해야 합니다. 계좌 개설 시에는 주민번호와 생년월일이 포함된 신분증이 필요하며, 학생증만 있는 경우 주민등록초본 등 추가 서류가 필요할 수 있습니다. 계좌 개설 후 인터넷뱅킹 신청을 진행하시면 됩니다. 가까운 OO은행 지점을 방문하여 필요한 서류를 확인하고 진행하시면 됩니다.'

## Case 3 - 공인인증서

In [ ]:
# 대화내역을 저장한 Memory 초기화하기
memory.clear()

In [ ]:
temp_df = df[df['대화셋일련번호'] == 'A28443']
temp_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
64830,금융/보험,사고 및 보상 문의,A28443,고객,1,OTP분실,,Q,안녕하세요 S은행이죠?,,,,은행,은행/ 금융기관,"은행,금융기관"
64831,금융/보험,사고 및 보상 문의,A28443,상담사,2,,OTP분실,A,,,,네 고객님 말씀하세요,,,
64832,금융/보험,사고 및 보상 문의,A28443,고객,3,OTP분실,,Q,다른게 아니구요 제가 지금 지갑을 잃어버려서요 OPT카드로 없어졌거든요?,,,,"지갑, OTP",OTP/일회용 비밀번호 생성기,"OTP,일회용비밀번호생성기"
64833,금융/보험,사고 및 보상 문의,A28443,상담사,4,,OTP분실,A,,,,네 많이 당황하셨겠습니다,,,
64834,금융/보험,사고 및 보상 문의,A28443,상담사,5,,OTP분실,A,,,,우선 고객님 인적사항 부터 확인후 안내도와드리겠습니다,"인저사항, 확인, 안내","확인/체크, 안내/설명","확인,설명"
64835,금융/보험,사고 및 보상 문의,A28443,상담사,6,,OTP분실,Q,,성함이요?,,,성함,성함/이름/성명,"성함,성명"
64836,금융/보험,사고 및 보상 문의,A28443,고객,7,OTP분실,,A,,,OOO,,OOO,,OOO
64837,금융/보험,사고 및 보상 문의,A28443,상담사,8,,OTP분실,Q,,주민번호앞자리 말씀해주세요?,,,주민번호,주민번호/주민등록번호,"주민번호,주민등록번호"
64838,금융/보험,사고 및 보상 문의,A28443,고객,9,OTP분실,,A,,,OOOOO입니다,,760210,,760210.0
64839,금융/보험,사고 및 보상 문의,A28443,상담사,10,,OTP분실,A,,,,네 잠시만 기다려주세요,,,


In [ ]:
llm_response = get_result_and_save_memory("안녕하세요 S은행이죠?")
llm_response

'네, 고객님. S은행입니다. 무엇을 도와드릴까요?'

In [ ]:
memory.load_memory_variables({})["history"]

'Human: 안녕하세요 S은행이죠?\nAI: 네, 고객님. S은행입니다. 무엇을 도와드릴까요?'

In [ ]:
llm_response = get_result_and_save_memory("다른게 아니구요 제가 지금 지갑을 잃어버려서요 OPT카드로 없어졌거든요?")
llm_response

'네, 많이 당황하셨겠습니다. 우선 고객님의 인적사항을 확인한 후 안내 도와드리겠습니다. 성함과 주민번호 앞자리를 말씀해 주시겠어요?'

In [ ]:
memory.load_memory_variables({})["history"]

'Human: 안녕하세요 S은행이죠?\nAI: 네, 고객님. S은행입니다. 무엇을 도와드릴까요?\nHuman: 다른게 아니구요 제가 지금 지갑을 잃어버려서요 OPT카드로 없어졌거든요?\nAI: 네, 많이 당황하셨겠습니다. 우선 고객님의 인적사항을 확인한 후 안내 도와드리겠습니다. 성함과 주민번호 앞자리를 말씀해 주시겠어요?'

In [ ]:
llm_response = get_result_and_save_memory("홍길동, 20010101입니다.")
llm_response

'네, 홍길동 고객님. 주민번호 앞자리를 확인했습니다. 잠시만 기다려 주세요.\n\n고객님께서 말씀하신 OPT카드는 00입출금 통장과 연계된 것으로 확인되었습니다. 재발급을 원하신다면, 본인이 직접 신분증을 지참하시고 가까운 영업점을 방문하시면 됩니다. 또한, 스마트뱅킹을 통해 모바일OTP도 발급 가능합니다. 추가로 궁금한 사항이 있으시면 언제든지 문의해 주세요. 감사합니다.'

In [ ]:
memory.load_memory_variables({})["history"]

'Human: 안녕하세요 S은행이죠?\nAI: 네, 고객님. S은행입니다. 무엇을 도와드릴까요?\nHuman: 다른게 아니구요 제가 지금 지갑을 잃어버려서요 OPT카드로 없어졌거든요?\nAI: 네, 많이 당황하셨겠습니다. 우선 고객님의 인적사항을 확인한 후 안내 도와드리겠습니다. 성함과 주민번호 앞자리를 말씀해 주시겠어요?\nHuman: 홍길동, 20010101입니다.\nAI: 네, 홍길동 고객님. 주민번호 앞자리를 확인했습니다. 잠시만 기다려 주세요.\n\n고객님께서 말씀하신 OPT카드는 00입출금 통장과 연계된 것으로 확인되었습니다. 재발급을 원하신다면, 본인이 직접 신분증을 지참하시고 가까운 영업점을 방문하시면 됩니다. 또한, 스마트뱅킹을 통해 모바일OTP도 발급 가능합니다. 추가로 궁금한 사항이 있으시면 언제든지 문의해 주세요. 감사합니다.'